# 不動産価格予測モデルに関する分析レポート (XGBoost版)

## 1. はじめに
本ノートブックでは、与えられた不動産データを用いて将来の住宅価格を予測するモデルを構築・評価する。  
本分析の目的は、データの特性を理解し、将来同様のデータが与えられた場合に高い汎用性を持った予測を行うことが可能なモデルを示すことである。  
今回は、ツリー系の勾配ブースティング手法である **XGBoost** による予測モデルを用いる。

## 2. データ概要
本分析では、以下の変数を含む不動産取引データを用いる。

- date: 売買成立日
- price: 売買価格(ドル) [予測対象]
- sqft_lot: 敷地面積
- sqft_living: 居住スペース面積
- grade: 建築・デザイン品質評価(1～13)
- lat: 緯度
- long: 経度
- yr_built: 建築年
- yr_renovated: 最終リノベーション年(0は未リノベーション)

分析の流れ:
1. データの読み込みと基本統計確認
2. 探索的データ分析(EDA)
3. 前処理・特徴量エンジニアリング
4. XGBoostモデルによる学習・評価
5. 結果考察・まとめ

In [2]:
# 必要ライブラリインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
import xgboost as xgb

%matplotlib inline
sns.set_style("whitegrid")


## 3. データ読み込みおよび基本確認
ここで`data.csv`を読み込み、基本的な情報を確認する。


In [4]:
# データ読み込み
# 'date' カラムを文字列として読み込み、その後日付型に変換
data = pd.read_csv("data.csv", dtype={'date': str})

# 'date' カラムを datetime 型に変換
data['date'] = pd.to_datetime(data['date'], format='%Y%m%d')

# データ概要の表示
display(data.head())
display(data.info())
display(data.describe().applymap(lambda x: f"{x:,.2f}"))


,date,price,sqft_lot,sqft_living,grade,lat,long,yr_built,yr_renovated
0,2014-10-13,221900,5650,1180,7,47.5112,-122.257,1955,0
1,2014-12-09,538000,7242,2570,7,47.7210,-122.319,1951,1991
2,2015-02-25,180000,10000,770,6,47.7379,-122.233,1933,0
3,2014-12-09,604000,5000,1960,7,47.5208,-122.393,1965,0
4,2015-02-18,510000,8080,1680,8,47.6168,-122.045,1987,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          21613 non-null  datetime64[ns]
 1   price         21613 non-null  int64         
 2   sqft_lot      21613 non-null  int64         
 3   sqft_living   21613 non-null  int64         
 4   grade         21613 non-null  int64         
 5   lat           21613 non-null  float64       
 6   long          21613 non-null  float64       
 7   yr_built      21613 non-null  int64         
 8   yr_renovated  21613 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(6)
memory usage: 1.5 MB


None

C:\Users\rainb\AppData\Local\Temp\ipykernel_5156\3066492741.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  display(data.describe().applymap(lambda x: f"{x:,.2f}"))


,date,price,sqft_lot,sqft_living,grade,lat,long,yr_built,yr_renovated
count,"21,613.00","21,613.00","21,613.00","21,613.00","21,613.00","21,613.00","21,613.00","21,613.00","21,613.00"
mean,",.2f","540,088.14","15,106.97","2,079.90",7.66,47.56,-122.21,"1,971.01",84.40
min,",.2f","75,000.00",520.00,290.00,1.00,47.16,-122.52,"1,900.00",0.00
25%,",.2f","321,950.00","5,040.00","1,427.00",7.00,47.47,-122.33,"1,951.00",0.00
50%,",.2f","450,000.00","7,618.00","1,910.00",7.00,47.57,-122.23,"1,975.00",0.00
75%,",.2f","645,000.00","10,688.00","2,550.00",8.00,47.68,-122.12,"1,997.00",0.00
max,",.2f","7,700,000.00","1,651,359.00","13,540.00",13.00,47.78,-121.31,"2,015.00","2,015.00"
std,nan,"367,127.20","41,420.51",918.44,1.18,0.14,0.14,29.37,401.68
